In [ ]:

def get_tables(file_name):
    '''
    This function can be used to download the tables that lives in S3 that are proccessed by the ETL workflow
    '''
    fs = s3fs.S3FileSystem() 
    source = f"s3://ift-bucket-jafet/historical/{file_name}.parquet"
    return pl.scan_parquet(source)



In [ ]:

#creating the table that will use for the model 

tot_market_share = market_share.filter(pl.col('MARKET_SHARE')>=1.0).group_by('K_GRUPO').agg(pl.col('MARKET_SHARE').sum())
maket_share_filtered_top8 = market_share.select('FECHA','K_GRUPO','MARKET_SHARE').filter(pl.col('MARKET_SHARE')>=1.0)
lineas_filtered_with_top8 = lineas_filtered.filter(pl.col('K_GRUPO').is_in(tot_market_share.select('K_GRUPO')))
traf_filtered_with_top8 = traf_filtered.filter(pl.col('K_GRUPO').is_in(tot_market_share.select('K_GRUPO')))

first_join = maket_share_filtered_top8.join(traf_filtered_with_top8, on=['FECHA','K_GRUPO'], how='left')
second_join = first_join.join(lineas_filtered_with_top8, on=['FECHA','K_GRUPO'], how='left')

sj_g003 = second_join.filter(pl.col('K_GRUPO')=='G003')

In [21]:
##This is only a preview of how to work with mlflow.

from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

categorical = []
numerical = ['L_NO_ESPECIFICADO_E','L_POSPAGOC_E','L_PREPAGO_E','L_TOTAL_E']
target = ['MARKET_SHARE']

df = sj_g003.select(numerical+categorical+target).with_columns(pl.col('L_NO_ESPECIFICADO_E').fill_null(0),pl.col('L_POSPAGOC_E').fill_null(0),pl.col('L_PREPAGO_E').fill_null(0),pl.col('L_TOTAL_E').fill_null(0))

mlflow.sklearn.autolog()

with mlflow.start_run():
    #separacion train test 80-20
    train,test = train_test_split(df)
    train,test = train_test_split(df)
    train_dicts = train.select(numerical+categorical).to_dicts()
    test_dicts = test.select(numerical+categorical).to_dicts()
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts)
    X_test = dv.fit_transform(test_dicts)
    y_train = train.select(target).to_numpy()
    y_test = test.select(target).to_numpy()
    lr = LinearRegression()
    lr.fit(X_train,y_train)
    y_pred = lr.predict(X_test)
    (rmse,mae,r2) = eval_metrics(y_test,y_pred)

2024/08/12 14:12:28 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/08/12 14:12:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run bold-ray-819 at: http://localhost:5000/#/experiments/0/runs/d7342c8213dc40f59f308a48c5bccdf9.
2024/08/12 14:12:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/0.
